In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
def load_distance_results(base_dir="distance_results"):
    """Load and combine all distance experiment results"""
    results = []
    
    # Walk through the directory structure
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith("_distance_summary.csv"):
                # Load the summary file
                filepath = os.path.join(root, file)
                df = pd.read_csv(filepath)
                results.append(df)
    
    # Combine all results
    if results:
        combined_df = pd.concat(results, ignore_index=True)
        return combined_df
    else:
        print("No results found!")
        return None

In [14]:
def analyze_distance_results(results_df=None):
    """Analyze the distance-based results with enhanced visualizations"""
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import matplotlib.font_manager as fm
    from matplotlib.ticker import LogLocator, LogFormatter
    
    if results_df is None:
        results_df = load_distance_results()
        
    if results_df is None or results_df.empty:
        print("No data to analyze!")
        return
    
    # Create output directory for plots
    plots_dir = "distance_analysis_plots"
    os.makedirs(plots_dir, exist_ok=True)
    
    # Set up beautiful typography
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['Montserrat', 'Arial', 'Helvetica', 'DejaVu Sans']
    plt.rcParams['font.size'] = 12
    plt.rcParams['axes.titlesize'] = 16
    plt.rcParams['axes.labelsize'] = 14
    plt.rcParams['xtick.labelsize'] = 11
    plt.rcParams['ytick.labelsize'] = 11
    plt.rcParams['legend.fontsize'] = 11
    plt.rcParams['figure.titlesize'] = 18
    
    # Create a beautiful color palette
    qec_colors = {
        'three_qubit_bit_flip': '#3498db',    # Blue
        'three_qubit_phase_flip': '#2ecc71',  # Green
        'five_qubit': '#e74c3c',              # Red
        'shor_nine': '#9b59b6',               # Purple
        'steane_seven': '#f39c12',            # Orange
        'none': '#7f8c8d'                     # Gray
    }
    
    # QEC method display names for legends
    qec_display_names = {
        'three_qubit_bit_flip': '3-Qubit Bit-Flip Code',
        'three_qubit_phase_flip': '3-Qubit Phase-Flip Code',
        'five_qubit': '5-Qubit Perfect Code',
        'shor_nine': 'Shor 9-Qubit Code',
        'steane_seven': 'Steane 7-Qubit Code',
        'none': 'No QEC'
    }
    
    # Different marker styles for methods
    markers = {
        'three_qubit_bit_flip': 'o',
        'three_qubit_phase_flip': 's',
        'five_qubit': 'D',
        'shor_nine': '^',
        'steane_seven': 'P',
        'none': 'X'
    }
    
    # Create a custom style
    sns.set_style("whitegrid", {
        'grid.linestyle': '--',
        'grid.alpha': 0.7,
        'axes.edgecolor': '#333333',
        'axes.linewidth': 1.2,
        'figure.facecolor': 'white'
    })
    
    # 1. QEC Method Comparison by Distance
    for error_type in results_df['error_type'].unique():
        for initial_state in results_df['initial_state'].unique():
            # Filter data
            data = results_df[(results_df['error_type'] == error_type) & 
                             (results_df['initial_state'] == initial_state)]
            
            if data.empty:
                continue
                
            plt.figure(figsize=(12, 8))
            
            for qec_method in data['qec_method'].unique():
                method_data = data[data['qec_method'] == qec_method]
                if method_data.empty:
                    continue
                    
                # Sort by distance for proper line connections
                method_data = method_data.sort_values(by='distance')
                
                plt.plot(method_data['distance'], 
                        method_data['avg_fidelity_with_qec'], 
                        marker=markers[qec_method],
                        markersize=9,
                        linestyle='-', 
                        linewidth=2.5,
                        label=qec_display_names[qec_method],
                        color=qec_colors[qec_method],
                        alpha=0.85)
                
                # Add data point labels
                for d, f in zip(method_data['distance'], method_data['avg_fidelity_with_qec']):
                    if f > 0.05:  # Only label if fidelity is significant
                        plt.annotate(f"{d} km", 
                                   (d, f),
                                   textcoords="offset points", 
                                   xytext=(0,7), 
                                   ha='center',
                                   fontsize=9,
                                   alpha=0.7)
            
            # Configure axes
            plt.xscale('log')
            plt.xlabel('Distance (km)', fontweight='bold')
            plt.ylabel('Average Fidelity', fontweight='bold')
            
            # Add minor grid lines to show distance detail
            plt.grid(True, which='minor', linestyle=':', alpha=0.3)
            plt.grid(True, which='major', linestyle='--', alpha=0.7)
            
            # Set axis limits
            plt.ylim(0, 1.05)
            
            # Add a descriptive title
            state_name = {"0": "|0⟩", "1": "|1⟩", "+": "|+⟩", "-": "|-⟩", 
                         "+i": "|+i⟩", "-i": "|-i⟩", "random": "Random"}
            error_name = error_type.replace("_", " ").title()
            
            # FIX: Convert initial_state to string before dictionary lookup
            plt.title(f'QEC Method Performance vs. Distance\n{state_name[str(initial_state)]} State with {error_name} Error',
                     fontweight='bold', pad=15)
            
            # Create a nicer legend
            legend = plt.legend(loc='best', frameon=True, fancybox=True, 
                              framealpha=0.95, edgecolor='#333333')
            legend.get_frame().set_linewidth(1.0)
            
            plt.tight_layout()
            
            # Save plot
            filename = f"{initial_state}_{error_type}_methods_comparison.png"
            plt.savefig(os.path.join(plots_dir, filename), dpi=300, bbox_inches='tight')
            plt.close()
    
    # 2. Error Mitigation Factor by Distance
    for error_type in results_df['error_type'].unique():
        for initial_state in results_df['initial_state'].unique():
            # Filter data
            data = results_df[(results_df['error_type'] == error_type) & 
                             (results_df['initial_state'] == initial_state) &
                             (results_df['qec_method'] != 'none')]
            
            if data.empty:
                continue
                
            plt.figure(figsize=(12, 8))
            
            for qec_method in data['qec_method'].unique():
                method_data = data[data['qec_method'] == qec_method]
                if method_data.empty:
                    continue
                    
                # Sort by distance
                method_data = method_data.sort_values(by='distance')
                
                plt.plot(method_data['distance'], 
                        method_data['avg_error_mitigation_factor'], 
                        marker=markers[qec_method],
                        markersize=9,
                        linestyle='-', 
                        linewidth=2.5,
                        label=qec_display_names[qec_method],
                        color=qec_colors[qec_method],
                        alpha=0.85)
                
                # Add data point labels
                for d, emf in zip(method_data['distance'], method_data['avg_error_mitigation_factor']):
                    plt.annotate(f"{d} km", 
                               (d, emf),
                               textcoords="offset points", 
                               xytext=(0,7), 
                               ha='center',
                               fontsize=9,
                               alpha=0.7)
            
            # Add threshold line
            plt.axhline(y=1.0, color='#e74c3c', linestyle='--', 
                       linewidth=2, label='No Improvement Threshold')
            
            # Configure axes
            plt.xscale('log')
            plt.xlabel('Distance (km)', fontweight='bold')
            plt.ylabel('Error Mitigation Factor', fontweight='bold')
            
            # Add minor grid lines
            plt.grid(True, which='minor', linestyle=':', alpha=0.3)
            plt.grid(True, which='major', linestyle='--', alpha=0.7)
            
            # Add a descriptive title
            state_name = {"0": "|0⟩", "1": "|1⟩", "+": "|+⟩", "-": "|-⟩", 
                         "+i": "|+i⟩", "-i": "|-i⟩", "random": "Random"}
            error_name = error_type.replace("_", " ").title()
            
            # Already fixed in original code
            plt.title(f'Error Mitigation Factor vs. Distance\n{state_name[str(initial_state)]} State with {error_name} Error',
                     fontweight='bold', pad=15)
            
            # Create a nicer legend
            legend = plt.legend(loc='best', frameon=True, fancybox=True, 
                              framealpha=0.95, edgecolor='#333333')
            legend.get_frame().set_linewidth(1.0)
            
            plt.tight_layout()
            
            # Save plot
            filename = f"{initial_state}_{error_type}_mitigation_factor.png"
            plt.savefig(os.path.join(plots_dir, filename), dpi=300, bbox_inches='tight')
            plt.close()
    
    # 3. QEC Effectiveness Threshold Analysis
    for qec_method in results_df['qec_method'].unique():
        if qec_method == 'none':
            continue
            
        for initial_state in results_df['initial_state'].unique():
            plt.figure(figsize=(12, 8))
            
            error_colors = {
                'amplitude_damping': '#e74c3c',   # Red
                'phase_damping': '#3498db'        # Blue
            }
            
            for error_type in results_df['error_type'].unique():
                # Filter data
                data = results_df[(results_df['error_type'] == error_type) & 
                                 (results_df['initial_state'] == initial_state) &
                                 (results_df['qec_method'] == qec_method)]
                
                if data.empty:
                    continue
                
                # Get values and sort by distance
                data = data.sort_values(by='distance')
                distances = data['distance'].values
                emfs = data['avg_error_mitigation_factor'].values
                
                # Plot data with finer styling
                label = error_type.replace("_", " ").title()
                plt.plot(distances, emfs, 
                        marker='o',
                        markersize=8,
                        linestyle='-', 
                        linewidth=2.5,
                        label=label,
                        color=error_colors.get(error_type, None))
                
                # Add data point labels
                for d, emf in zip(distances, emfs):
                    plt.annotate(f"{d} km", 
                               (d, emf),
                               textcoords="offset points", 
                               xytext=(0,7), 
                               ha='center',
                               fontsize=9,
                               alpha=0.7)
                
                # Find threshold if it exists and add a vertical line
                if any(emfs < 1.0) and any(emfs >= 1.0):
                    # Find crossover points
                    for i in range(len(emfs)-1):
                        if (emfs[i] >= 1.0 and emfs[i+1] < 1.0) or (emfs[i] < 1.0 and emfs[i+1] >= 1.0):
                            x1, y1 = distances[i], emfs[i]
                            x2, y2 = distances[i+1], emfs[i+1]
                            threshold = x1 + (x2 - x1) * (1.0 - y1) / (y2 - y1)
                            
                            plt.axvline(x=threshold, linestyle='--', alpha=0.7, 
                                       linewidth=1.5,
                                       color=error_colors.get(error_type, 'gray'))
                            
                            # Add an annotation for the threshold
                            plt.annotate(f"{label} threshold:\n{threshold:.1f} km",
                                        xy=(threshold, 1),
                                        xytext=(threshold*1.2, 1.2),
                                        arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0.2"),
                                        bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
                                        ha='center', va='center')
            
            # Add threshold line
            plt.axhline(y=1.0, color='#e74c3c', linestyle='--', 
                       linewidth=2, label='Effectiveness Threshold')
            
            # Configure axes
            plt.xscale('log')
            plt.xlabel('Distance (km)', fontweight='bold')
            plt.ylabel('Error Mitigation Factor', fontweight='bold')
            
            # Add minor grid lines
            plt.grid(True, which='minor', linestyle=':', alpha=0.3)
            plt.grid(True, which='major', linestyle='--', alpha=0.7)
            
            # Set y-axis limits to better show the threshold region
            plt.ylim(0, plt.ylim()[1]*1.1)
            
            # Add a descriptive title
            state_name = {"0": "|0⟩", "1": "|1⟩", "+": "|+⟩", "-": "|-⟩", 
                         "+i": "|+i⟩", "-i": "|-i⟩", "random": "Random"}
            method_name = qec_display_names[qec_method]
            
            # FIX: Convert initial_state to string before dictionary lookup
            plt.title(f'QEC Effectiveness Threshold Analysis\n{state_name[str(initial_state)]} State with {method_name}',
                     fontweight='bold', pad=15)
            
            # Create a nicer legend
            legend = plt.legend(loc='best', frameon=True, fancybox=True, 
                              framealpha=0.95, edgecolor='#333333')
            legend.get_frame().set_linewidth(1.0)
            
            plt.tight_layout()
            
            # Save plot
            filename = f"{initial_state}_{qec_method}_threshold_analysis.png"
            plt.savefig(os.path.join(plots_dir, filename), dpi=300, bbox_inches='tight')
            plt.close()
    
    # 4. Distance Scaling Analysis (Matrix Plot)
    error_types = results_df['error_type'].unique()
    initial_states = results_df['initial_state'].unique()
    
    fig, axes = plt.subplots(len(error_types), len(initial_states), 
                          figsize=(16, 12), sharex=True)
    
    # Add a super title
    fig.suptitle('Quantum Error Correction Performance Across Initial States and Error Types', 
               fontsize=18, fontweight='bold', y=0.98)
    
    # State and error type display names
    state_name = {"0": "|0⟩", "1": "|1⟩", "+": "|+⟩", "-": "|-⟩", 
                 "+i": "|+i⟩", "-i": "|-i⟩", "random": "Random"}
    
    error_name = {
        "amplitude_damping": "Amplitude Damping",
        "phase_damping": "Phase Damping"
    }
    
    for i, error_type in enumerate(error_types):
        for j, initial_state in enumerate(initial_states):
            # Get the correct axis
            if len(error_types) > 1 and len(initial_states) > 1:
                ax = axes[i, j]
            elif len(error_types) > 1:
                ax = axes[i]
            elif len(initial_states) > 1:
                ax = axes[j]
            else:
                ax = axes
            
            # Filter data
            data = results_df[(results_df['error_type'] == error_type) & 
                             (results_df['initial_state'] == initial_state)]
            
            if data.empty:
                ax.text(0.5, 0.5, 'No Data', 
                       ha='center', va='center', 
                       transform=ax.transAxes)
                continue
            
            # Different methods with colors and markers
            for qec_method in data['qec_method'].unique():
                method_data = data[data['qec_method'] == qec_method]
                if method_data.empty or len(method_data) < 2:
                    continue
                
                # Sort by distance
                method_data = method_data.sort_values(by='distance')
                
                ax.plot(method_data['distance'], 
                       method_data['avg_fidelity_with_qec'], 
                       marker=markers[qec_method],
                       markersize=6,
                       linestyle='-', 
                       linewidth=2,
                       label=qec_display_names[qec_method],
                       color=qec_colors[qec_method],
                       alpha=0.85)
            
            # Add detailed title
            error_display = error_name.get(error_type, error_type.replace("_", " ").title())
            # FIX: Convert initial_state to string before dictionary lookup
            ax.set_title(f"{state_name[str(initial_state)]}, {error_display}", fontsize=12)
            
            # Configure axis
            ax.set_xscale('log')
            ax.grid(True, which='major', linestyle='--', alpha=0.5)
            ax.grid(True, which='minor', linestyle=':', alpha=0.3)
            
            # Only add labels on edges
            if i == len(error_types) - 1:
                ax.set_xlabel('Distance (km)', fontsize=11)
            if j == 0:
                ax.set_ylabel('Fidelity', fontsize=11)
            
            # Set y-axis limits
            ax.set_ylim(0, 1.05)
            
            # Add legend only on the first subplot
            if i == 0 and j == 0:
                ax.legend(loc='lower left', fontsize=9, frameon=True, 
                         fancybox=True, framealpha=0.9)
    
    # Adjust layout
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    
    # Save the matrix plot
    plt.savefig(os.path.join(plots_dir, "distance_scaling_matrix.png"), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Analysis complete! Enhanced visualizations saved to: {plots_dir}")


In [15]:
if __name__ == "__main__":
    analyze_distance_results()

Analysis complete! Enhanced visualizations saved to: distance_analysis_plots
